In [ ]:
%pip install ipywidgets
%pip install rapidfuzz
%pip install tldextract
%pip install scikit-learn
%pip install -U ipywidgets jupyterlab_widgets

In [3]:
# ===============================
# ZIA THREAT HUNT WORKBENCH
# CELL 1: DATA PREP ONLY
# ===============================

import pandas as pd
import numpy as np
import tldextract
from rapidfuzz import fuzz
from sklearn.cluster import KMeans

LA_TZ = "America/Los_Angeles"
ZIA_FMT = "%B %d, %Y %I:%M:%S %p"

# -------------------------------
# 1. Load Dataset
# -------------------------------
FILE_PATH = "../data/lab_dprk.csv"

df = pd.read_csv(FILE_PATH, low_memory=False)
print("Rows:", len(df))

# -------------------------------
# 2. Parse PST Time Fields
# -------------------------------
def parse_zia_pst(series: pd.Series) -> pd.Series:
    s = series.astype("string")
    s = s.str.replace(r"\s+PST$", "", regex=True)
    dt = pd.to_datetime(s, format=ZIA_FMT, errors="coerce")
    return dt.dt.tz_localize(LA_TZ, ambiguous="NaT", nonexistent="shift_forward")

df["Logged Time Parsed"] = parse_zia_pst(df["Logged Time"])
df["Event Time Parsed"]  = parse_zia_pst(df["Event Time"])

# -------------------------------
# 3. Extract Hostname + TLD
# -------------------------------
def extract_domain(url):
    if pd.isna(url):
        return None, None
    ext = tldextract.extract("http://" + url)
    domain = f"{ext.domain}.{ext.suffix}" if ext.suffix else ext.domain
    return domain, ext.suffix

df[["Hostname", "TLD"]] = df["URL"].apply(
    lambda x: pd.Series(extract_domain(x))
)

# -------------------------------
# 4. Majestic Million Enrichment
# -------------------------------
majestic = pd.read_csv("../data/majestic_million.csv")
majestic_set = set(majestic["Domain"].str.lower())

df["In Majestic"] = df["Hostname"].str.lower().isin(majestic_set)

# -------------------------------
# 5. Simple Clustering (Optional)
# -------------------------------
cluster_df = df[["Received Bytes","Sent Bytes","Proxy Latency (ms)"]].fillna(0)

if len(cluster_df) > 0:
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    df["Cluster"] = kmeans.fit_predict(cluster_df)

print("Data preparation complete.")
df.head()


Rows: 8080
Data preparation complete.


,No.,Logged Time,Event Time,User,SSL Inspected,URL,Policy Action,Cloud Application Class,Cloud Application,Application Status,...,Client Digital Signature Proposal,Download User-Defined File Type,Upload User-Defined File Type,Extranet Resource,Logged Time Parsed,Event Time Parsed,Hostname,TLD,In Majestic,Cluster
0,1,"January 27, 2026 2:30:23 PM PST","January 27, 2026 2:30:23 PM PST",karen@acme.zscaler.net,No,ssl.gstatic.com,Dropped due to failed client SSL handshake,General Browsing,NaN,NaN,...,"[""Classical""]",NaN,NaN,NaN,2026-01-27 14:30:23-08:00,2026-01-27 14:30:23-08:00,gstatic.com,com,True,0
1,2,"January 27, 2026 2:30:23 PM PST","January 27, 2026 2:30:23 PM PST",karen@acme.zscaler.net,Yes,self.events.data.microsoft.com/onecollector/1.0/,Allowed,General Browsing,NaN,NaN,...,"[""Classical""]",NaN,NaN,NaN,2026-01-27 14:30:23-08:00,2026-01-27 14:30:23-08:00,microsoft.com,com,True,0
2,3,"January 27, 2026 2:30:23 PM PST","January 27, 2026 2:30:23 PM PST",karen@acme.zscaler.net,No,ssl.gstatic.com,Dropped due to failed client SSL handshake,General Browsing,NaN,NaN,...,"[""Classical""]",NaN,NaN,NaN,2026-01-27 14:30:23-08:00,2026-01-27 14:30:23-08:00,gstatic.com,com,True,0
3,4,"January 27, 2026 2:30:23 PM PST","January 27, 2026 2:30:23 PM PST",karen@acme.zscaler.net,No,detectportal.firefox.com/canonical.html,Allowed,General Browsing,NaN,NaN,...,NaN,NaN,NaN,NaN,2026-01-27 14:30:23-08:00,2026-01-27 14:30:23-08:00,firefox.com,com,True,0
4,5,"January 27, 2026 2:30:23 PM PST","January 27, 2026 2:30:23 PM PST",karen@acme.zscaler.net,No,detectportal.firefox.com/success.txt?ipv4,Allowed,General Browsing,NaN,NaN,...,NaN,NaN,NaN,NaN,2026-01-27 14:30:23-08:00,2026-01-27 14:30:23-08:00,firefox.com,com,True,0


In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.express as px
import pandas as pd

LA_TZ = "America/Los_Angeles"

# ---------- Controls ----------
pivot_dropdown = widgets.Dropdown(
    options=sorted(df.columns),
    value="Hostname" if "Hostname" in df.columns else df.columns[0],
    description="Pivot:",
    layout=widgets.Layout(width="500px")
)

top_n_slider = widgets.IntSlider(
    value=30, min=5, max=200, step=5,
    description="Top N:",
    continuous_update=False
)

start_picker = widgets.DatetimePicker(description="Start (LA)")
end_picker   = widgets.DatetimePicker(description="End (LA)")

search_box = widgets.Text(
    description="Search:",
    placeholder="e.g. github, /dns-query, POST, user@acme.com",
    layout=widgets.Layout(width="700px")
)

search_cols = widgets.SelectMultiple(
    options=["(all columns)", "URL", "Hostname", "User", "User Agent", "Request Method", "Policy Action",
             "Client IP", "Client External IP", "Server IP", "Referrer URL", "Download File Type",
             "Download File Name", "URL Category", "Threat Name"],
    value=("Hostname", "URL"),   # default search scope
    description="Search in:",
    layout=widgets.Layout(width="700px", height="160px")
)

apply_btn = widgets.Button(description="Apply Filters", button_style="primary")
reset_btn = widgets.Button(description="Reset", button_style="warning")

# ---------- Outputs ----------
pivot_output = widgets.Output()
tail_output  = widgets.Output()
table_output = widgets.Output()
status_out   = widgets.Output()

# ---------- Helpers ----------
def _ensure_la(ts):
    """Coerce a python datetime or pandas Timestamp to tz-aware LA Timestamp."""
    if ts is None:
        return None
    t = pd.Timestamp(ts)
    if t.tzinfo is None:
        return t.tz_localize(LA_TZ)
    return t.tz_convert(LA_TZ)

def filtered_df():
    d = df

    # Time filter (Logged Time Parsed)
    start = _ensure_la(start_picker.value)
    end   = _ensure_la(end_picker.value)

    if start is not None and end is not None:
        # auto-swap if user picks start > end
        if start > end:
            start, end = end, start
        d = d[(d["Logged Time Parsed"] >= start) & (d["Logged Time Parsed"] <= end)]

    # Search filter
    q = (search_box.value or "").strip()
    if q:
        cols = list(search_cols.value)
        if "(all columns)" in cols:
            cols = list(d.columns)

        mask = pd.Series(False, index=d.index)
        for c in cols:
            if c in d.columns:
                mask = mask | d[c].astype("string").fillna("").str.contains(q, case=False, regex=False)
        d = d[mask]

    return d

def pivot_count_df(dataframe, column):
    return (
        dataframe[column]
        .astype("string").fillna("NaN")
        .value_counts()
        .rename_axis(column)
        .reset_index(name="Count")
    )

# ---- Filtered table columns (requested) ----
TABLE_COLS = [
    "Event Time",
    "User",
    "URL",
    "URL Category",
    "Policy Action",
    "Application Status",   # will be shown as "Status"
    "User Agent",
    "Client IP",
    "Client External IP",
    "Server IP",
    "Request Method",
    "Referrer URL",
    "Download File Type",
    "Download File Name",
]

def render_filtered_table(d, limit=200):
    cols = [c for c in TABLE_COLS if c in d.columns]
    out = d.loc[:, cols].copy()

    # Rename to match requested header wording
    out = out.rename(columns={"Application Status": "Status"})

    # Sort by Event Time if parsed column exists
    if "Event Time Parsed" in d.columns:
        out = out.assign(_sort=d["Event Time Parsed"]).sort_values("_sort").drop(columns=["_sort"])

    display(out.head(limit))

def render_all(*_):
    d = filtered_df()

    with status_out:
        clear_output()
        print(f"Filtered rows: {len(d):,} / {len(df):,}")

    with pivot_output:
        clear_output()
        out = pivot_count_df(d, pivot_dropdown.value).head(top_n_slider.value)
        display(out)

    with tail_output:
        clear_output()
        if "Hostname" in d.columns:
            freq = (
                d["Hostname"]
                .astype("string").fillna("NaN")
                .value_counts()
                .rename_axis("Hostname")
                .reset_index(name="Count")
            )
            tail = freq.tail(30)
            fig = px.bar(
                tail,
                y="Hostname",
                x="Count",
                orientation="h",
                title="Long Tail Domains (Least Frequent) — Filtered View"
            )
            fig.show()
        else:
            print("No Hostname column found.")

    with table_output:
        clear_output()
        render_filtered_table(d, limit=200)

def do_reset(_):
    start_picker.value = None
    end_picker.value = None
    search_box.value = ""
    search_cols.value = ("Hostname", "URL")
    render_all()

# ---------- Wire events ----------
apply_btn.on_click(lambda _: render_all())
reset_btn.on_click(do_reset)

pivot_dropdown.observe(render_all, names="value")
top_n_slider.observe(render_all, names="value")
start_picker.observe(render_all, names="value")
end_picker.observe(render_all, names="value")
search_box.observe(render_all, names="value")
search_cols.observe(render_all, names="value")

# ---------- Layout ----------
ui = widgets.VBox([
    widgets.HTML("<h2>ZIA Threat Hunting Workbench</h2>"),

    widgets.HTML("<b>Pivot Explorer (filtered)</b>"),
    widgets.HBox([pivot_dropdown, top_n_slider]),
    status_out,
    pivot_output,

    widgets.HTML("<b>Time Filters (Logged Time Parsed, LA)</b>"),
    widgets.HBox([start_picker, end_picker]),

    widgets.HTML("<b>Search</b>"),
    search_box,
    search_cols,
    widgets.HBox([apply_btn, reset_btn]),

    tail_output,

    widgets.HTML("<b>Filtered Transactions (table view)</b>"),
    table_output
])

display(ui)
render_all()
